In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#IMPORT WAITING LIST
url = r'C:\Users\achie\Github\NHS-Surgery\V1'

url = url + r'\Input.xlsx'
Patient_List = pd.read_excel(url, sheet_name = 0, header=0, index_col=0)
Theatres = pd.read_excel(url, sheet_name = 1, header=0, index_col=0)

%store -r NDays

start = dt.datetime(2020,12,28)
end = start + dt.timedelta(days=NDays)

In [3]:
#GET NEW WAITING TIME

today = dt.datetime.today()

Patient_List['Waiting_time'] = (today - Patient_List['Date of Admission']).dt.days

In [4]:
#OPERATION TIME BY WAITING URGENCY

Patient_List['Waiting_efficiency'] = (Patient_List['Waiting_time'] / 
                                       Patient_List['Expected length of operation (mins)'])

Waiting_Efficiency = Patient_List.sort_values(by='Waiting_efficiency', ascending=False)

In [5]:
#SPLIT SPECIALTY TYPES

List_of_data = [Waiting_Efficiency[Waiting_Efficiency['Specialty Descriptions'] == name] 
                    for name in Waiting_Efficiency['Specialty Descriptions'].unique()]

Overall_slots = pd.DataFrame()

Total = len(List_of_data)

for type in range(0, Total):
    
    Surgery = List_of_data[type].copy()

    Surgery['Total_time'] = Surgery['Expected length of operation (mins)'].cumsum()

    Surgery['Specialty_rank'] = Surgery['Total_time'] / 240
    Surgery['Specialty_rank'] = Surgery['Specialty_rank'].apply(np.ceil)
    
    Slot_operating_time = Surgery.groupby("Specialty_rank")['Expected length of operation (mins)'].sum()
      
    
    Slot_ranking = Surgery.groupby("Specialty_rank")['Hospital Urgency Score'].sum()

    No_operations = Surgery.groupby("Specialty_rank")['Hospital Urgency Score'].count()
    
    Wait_time = Surgery.groupby("Specialty_rank")['Waiting_time'].sum()

    Slot_ranking = pd.concat([Slot_ranking, No_operations, Wait_time], axis=1)

    Type = Surgery['Specialty Descriptions'].unique()
    Type = Type[0]
    Slot_ranking['Type'] = Type

    Slot_ranking.columns = ['Total_urgency', 'No_operations', 'Total_wait', 'Specialty']
    
    Slot_ranking = Slot_ranking.round({'Total_wait': 2})
    
    Slot_ranking = Slot_ranking.sort_values(by='Total_wait', ascending=False)
    Slot_ranking = Slot_ranking.reset_index()         
    
    Overall_slots = Overall_slots.append(Slot_ranking)
        
Overall_slots = Overall_slots.sort_values(by='Total_wait', ascending=False)

Overall_slots = Overall_slots.reset_index()
Overall_slots['Theatre_rank'] = Overall_slots.index

In [6]:
#CREATE SCHEDULE

Schedule = pd.DataFrame()

Type_fill = Overall_slots['Specialty'].unique()
Type_fill = pd.DataFrame(Type_fill)
Type_fill.columns = ['Specialty']
Type_fill.index = Type_fill['Specialty']

Overall_count = 0

Overall_slots['Session'] = "None"

def daterange(start, end):
    for n in range(int((end - start).days)):
        yield start + dt.timedelta(n)       

for date in daterange(start, end):
    weekday = date.strftime('%A')
    day_num = date.weekday() + 1

    for half_day in range(0,2):
        rank_slot = 0
        Updated_slots = Overall_slots[Overall_slots['Session'] == "None"]
        Updated_slots = Updated_slots.reset_index(drop=True)  

        
        slot = (day_num * 2) - 1 + half_day

        theatre_no = 1
        no_slots = 0
        Type_fill['No_slots'] = 0
        
        while theatre_no <= 9:
            if rank_slot + 1 > len(Updated_slots)-1:
                theatre_no = theatre_no + 1
                continue
                
            Type = Updated_slots.loc[rank_slot, 'Specialty']
            Availability = Theatres.loc[Type, slot]
            no_slots = Type_fill.loc[Type, 'No_slots']
                
            if no_slots >= Availability:
                rank_slot = rank_slot + 1
                
            else:
                Schedule.loc[Overall_count, 'Date'] = date
                Schedule.loc[Overall_count, 'Day'] = weekday
                Schedule.loc[Overall_count, 'Session'] = half_day
                Schedule.loc[Overall_count, 'Theatre'] = theatre_no
                Schedule.loc[Overall_count, 'Speciality'] = Type
                Schedule.loc[Overall_count, 'Operations'] = Updated_slots.loc[rank_slot, 'No_operations']
                Schedule.loc[Overall_count, 'Clinical_urgency'] = Updated_slots.loc[rank_slot, 'Total_urgency']
                Schedule.loc[Overall_count, 'Waiting_time'] = Updated_slots.loc[rank_slot, 'Total_wait']

                Ref = Updated_slots.loc[rank_slot, 'Theatre_rank'] 

                Overall_slots.loc[Ref, 'Date'] = date
                Overall_slots.loc[Ref, 'Day'] = weekday
                Overall_slots.loc[Ref, 'Session'] = half_day

                rank_slot = rank_slot + 1
                theatre_no = theatre_no + 1
                no_slots = no_slots + 1
                Type_fill.loc[Type, 'No_slots'] = no_slots
                
                Overall_count = Overall_count + 1

In [7]:
#EFFICIENCY SUMMARY

No_patients = Schedule['Operations'].sum()
Total_clinical = Schedule['Clinical_urgency'].sum()
Total_waiting = Schedule['Waiting_time'].sum()
Total_waiting = round(Total_waiting, 2)
Avg_clinical = Total_clinical / No_patients
Avg_clinical = round(Avg_clinical, 1)
Avg_wait = Total_waiting / No_patients
Avg_wait = round(Avg_wait, 0)

TNo_patients = len(Patient_List)
TTotal_clinical = Patient_List['Hospital Urgency Score'].sum()
TTotal_waiting = Patient_List['Waiting_time'].sum()
TAvg_clinical = Patient_List['Hospital Urgency Score'].mean()
TAvg_clinical = round(TAvg_clinical,1)
TAvg_waiting = Patient_List['Waiting_time'].mean()
TAvg_waiting = round(TAvg_waiting,0)

ANo_patients = TNo_patients - No_patients
ATotal_clinical = TTotal_clinical - Total_clinical
ATotal_waiting = TTotal_waiting - Total_waiting
AAvg_clinical = ATotal_clinical / ANo_patients
AAvg_clinical = round(AAvg_clinical,1)
AAvg_waiting = ATotal_waiting / ANo_patients
AAvg_waiting = round(AAvg_waiting,0)

Efficiency_Summary = ['Patients Scheduled', 'Total Clinical Urgency', 'Average Clinical Urgency',
                      'Total Waiting Time (days)', 'Average waiting time (days)']
Efficiency_Summary = pd.DataFrame(Efficiency_Summary)

Efficiency_Summary.columns = ['Summary']
Efficiency_Summary.index = Efficiency_Summary['Summary']
Efficiency_Summary = Efficiency_Summary.drop('Summary', 1)

#OPERATIONS
Efficiency_Summary.loc['Patients Scheduled', 'Operated_on'] = No_patients
Efficiency_Summary.loc['Total Clinical Urgency', 'Operated_on'] = Total_clinical
Efficiency_Summary.loc['Average Clinical Urgency', 'Operated_on'] = Avg_clinical
Efficiency_Summary.loc['Total Waiting Time (days)', 'Operated_on'] = Total_waiting
Efficiency_Summary.loc['Average waiting time (days)', 'Operated_on'] = Avg_wait

#BEFORE
Efficiency_Summary.loc['Patients Scheduled', 'Before'] = TNo_patients
Efficiency_Summary.loc['Total Clinical Urgency', 'Before'] = TTotal_clinical
Efficiency_Summary.loc['Average Clinical Urgency', 'Before'] = TAvg_clinical
Efficiency_Summary.loc['Total Waiting Time (days)', 'Before'] = TTotal_waiting
Efficiency_Summary.loc['Average waiting time (days)', 'Before'] = TAvg_waiting

#AFTER
Efficiency_Summary.loc['Patients Scheduled', 'After'] = ANo_patients
Efficiency_Summary.loc['Total Clinical Urgency', 'After'] = ATotal_clinical
Efficiency_Summary.loc['Average Clinical Urgency', 'After'] = AAvg_clinical
Efficiency_Summary.loc['Total Waiting Time (days)', 'After'] = ATotal_waiting
Efficiency_Summary.loc['Average waiting time (days)', 'After'] = AAvg_waiting


PDiff = ANo_patients - TNo_patients
CDiff = AAvg_clinical - TAvg_clinical
WDiff = AAvg_waiting - TAvg_waiting

PDiff = round(PDiff,0)
CDiff = round(CDiff,1)
WDiff = round(WDiff,0)

In [8]:
url = r'C:\Users\achie\Github\NHS-Surgery\V1\Schedule - Waiting Efficiency.xlsx'
writer = pd.ExcelWriter(url , engine='xlsxwriter')
Efficiency_Summary.to_excel(writer, sheet_name='Summary')
Waiting_Efficiency.to_excel(writer, sheet_name='Patients by Waiting Efficiency')
Overall_slots.to_excel(writer, sheet_name='Specialty Slot Order')
Schedule.to_excel(writer, sheet_name='Theatre Schedule')
writer.save()

In [10]:
#SUMMARY STATISTICS

print("If you schedule patients by WAITING TIME over next " + str(NDays) + " days")
print("Patients scheduled:          " + str(No_patients))
print("  - Average urgency:         " + str(Avg_clinical))
print("  - Average waiting:         " + str(Avg_wait))
print(" ")
print("Waiting list after:")
print("Patients:                    " + str(ANo_patients) +  "  (" + str(PDiff) + ")") 
print("  - Average urgency:         " + str(AAvg_clinical)+  "   (" + str(CDiff) + ")") 
print("  - Average waiting:         " + str(AAvg_waiting) +  "  (" + str(WDiff) + ")") 
print("--------------")
print(" ")

If you schedule patients by WAITING TIME over next 7 days
Patients scheduled:          531.0
  - Average urgency:         20.4
  - Average waiting:         296.0
 
Waiting list after:
Patients:                    649.0  (-531.0)
  - Average urgency:         21.9   (0.7)
  - Average waiting:         253.0  (-20.0)
--------------
 
